In [2]:
#importing required packages 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, HuberRegressor, ElasticNet, BayesianRidge
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
import time

In [3]:
#loading the data and setting my features and target variables in the training data 
trainh = pd.read_csv('train (1).csv', sep=',')
testh = pd.read_csv('test (1).csv', sep=',')
features, target = trainh.iloc[:,:-1], trainh.iloc[:,-1]
combineddf = pd.concat([trainh, testh], axis = 0)
combineddf.index = list(range(0,2919))
combineddf

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,NaN
2915,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,NaN
2916,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,NaN
2917,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,NaN


In [13]:
str5 = 'RandomForestRegressor 60 0.2 -0.2'
if str5[0:21] == 'RandomForestRegressor':
    print(True)
else:
    print(False)

True


In [28]:
dict5 = {'key1':1,'key2':2}
dict((sorted(dict5.items(), key = lambda x: x[0][1], reverse = False)))

{'key1': 1, 'key2': 2}

In [36]:
class Feature_Prep(object):
    
    def __init__(self):
        self.featuredict = {}
        self.objectlist = []
        self.numlist = []
        self.df1 = pd.DataFrame(np.repeat(0,1460))
        self.trainingdf = None
        self.testingdf = None
        self.dict1 = {}
        self.trainingsubset = None
        self.testingsubset = None
        self.model = None 
        self.x_train = None          
        self.x_test = None           
        self.y_train = None          #y_train of target
        self.y_test = None           #y_test of target
        self.target = None
        self.models = ['Ridge', 'Lasso', 'ElasticNet', 'BayesianRidge', 'HuberRegressor', 'AdaBoostRegressor', 'GradientBoostingRegressor', 'RandomForestRegressor']
        self.results_dict1 = {}
        self.results_dict2 = {}
        self.feature_name_list = []
        self.dict_ridge = {}
        self.dict_lasso = {}
        self.dict_elasticnet = {}
        self.dict_bayesianridge = {}
        self.dict_huberregressor = {}
        self.dict_adaboost = {}
        self.dict_gradientboosting = {}
        self.dict_randomforest = {}
        self.masterlist = []
        
    def type_filter(self, X):    #X is your trainh dataset
        for idx, x in enumerate(X.dtypes):
            self.featuredict[X.columns[idx]] = x 
        for i in list(filter(lambda x: x[1] == 'O', self.featuredict.items())):
            self.objectlist.append(i[0])
        return self.objectlist
            
    def dummify(self, X, model, posinterval, negcutoff):       #X is your combineddf dataset
        self.target = X.iloc[0:1460, -1]
        self.type_filter(X)
        for feature in self.objectlist:
            self.df1 = pd.concat([self.df1, pd.get_dummies(X[feature], prefix = feature, drop_first=True)], axis = 1)
        for i in list(filter(lambda x: x[1] == 'int64' or x[1] == 'float64', self.featuredict.items())):
            try:
                self.df1 = pd.concat([self.df1, X[i[0]]], axis = 1)
            except:
                continue
        self.trainingdf = self.df1.iloc[0:1460,1:]
        self.testingdf = self.df1.iloc[1460:,1:-1]
        self.trainingdf = self.trainingdf.fillna(self.trainingdf.mean())
        self.testingdf = self.testingdf.fillna(self.testingdf.mean())
        for model in self.models:
            self.set_model(model)
            for cutoff in posinterval:
                dict1 = {}
                for idx, i in enumerate(self.trainingdf.corr().iloc[:,-1]):
                    if i > cutoff or i < negcutoff:
                        dict1[self.trainingdf.columns[idx]] = i
                    else:
                        continue
                self.feature_name_list = [x[0] for x in dict1.items() if x[0] != 'SalePrice']
                self.trainingsubset = self.trainingdf[self.feature_name_list]
                self.testingsubset = self.testingdf[self.feature_name_list]

                self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.trainingsubset, self.target, test_size = 0.2, random_state = 42)

                self.model.fit(self.x_train, self.y_train)
                key1 = model + ' ' + str(len(self.feature_name_list))
                self.results_dict1[key1] = (np.round(self.model.score(self.x_train, self.y_train),4), np.round(self.model.score(self.x_test, self.y_test),4), model, len(self.feature_name_list), np.round(cutoff,2))
        
        print(len(self.results_dict1))
        return sorted(self.results_dict1.items(), key = lambda x: (x[1][2],x[1][1]), reverse = True)
#         for k, v in self.results_dict1.items():
#             if k[0:5] == 'Ridge':
#                 try:
#                     self.dict_ridge[k] = v 
#                 except:
#                     continue
#             if k[0:5] == 'Lasso':
#                 try:
#                     self.dict_lasso[k] = v  
#                 except:
#                     continue
#             if k[0:10] == 'ElasticNet':
#                 try:
#                     self.dict_elasticnet[k] = v
#                 except:
#                     continue
#             if k[0:14] == 'HuberRegressor':
#                 try:
#                     self.dict_huberregressor[k] = v
#                 except:
#                     continue
#             if k[0:13] == 'BayesianRidge':
#                 try:
#                     self.dict_bayesianridge[k] = v
#                 except:
#                     continue
#             if k[0:17] == 'AdaBoostRegressor':
#                 try:
#                     self.dict_adaboost[k] = v
#                 except:
#                     continue
#             if k[0:25] == 'GradientBoostingRegressor':
#                 try:
#                     self.dict_gradientboosting[k] = v
#                 except:
#                     continue
#             if k[0:21] == 'RandomForestRegressor':
#                 try:
#                     self.dict_randomforest[k] == v
#                 except:
#                     continue
                
#         self.dict_ridge = dict(sorted(self.dict_ridge.items(), key = lambda x: x[1][1], reverse = True))
#         self.dict_lasso = dict(sorted(self.dict_lasso.items(), key = lambda x: x[1][1], reverse = True))
#         self.dict_elasticnet = dict(sorted(self.dict_elasticnet.items(), key = lambda x: x[1][1], reverse = True))
#         self.dict_huberregressor = dict(sorted(self.dict_huberregressor.items(), key = lambda x: x[1][1], reverse = True))
#         self.dict_bayesianridge = dict(sorted(self.dict_bayesianridge.items(), key = lambda x: x[1][1], reverse = True))
#         self.dict_adaboost = dict(sorted(self.dict_adaboost.items(), key = lambda x: x[1][1], reverse = True))
#         self.dict_gradientboosting = dict(sorted(self.dict_gradientboosting.items(), key = lambda x: x[1][1], reverse = True))
#         self.dict_randomforest = dict(sorted(self.dict_randomforest.items(), key = lambda x: x[1][1], reverse = True))
        
    def best_features(self, X, model, posinterval, negcutoff):
        for i in range(0,4):
            self.dummify(X, model, posinterval, negcutoff)
            self.masterlist.append(dict(list(self.dict_ridge.items())[0:1]))
            self.masterlist.append(dict(list(self.dict_lasso.items())[0:1]))
            self.masterlist.append(dict(list(self.dict_elasticnet.items())[0:1]))
            self.masterlist.append(dict(list(self.dict_huberregressor.items())[0:1]))
            self.masterlist.append(dict(list(self.dict_bayesianridge.items())[0:1]))
            self.masterlist.append(dict(list(self.dict_adaboost.items())[0:1]))
            self.masterlist.append(dict(list(self.dict_gradientboosting.items())[0:1]))
            self.masterlist.append(dict(list(self.dict_randomforest.items())[0:1]))
        print(self.masterlist)
        pass

#                 print(sorted(self.dict_ridge.items(), key = lambda x: x[1][1], reverse = True))
#                 print(sorted(self.dict_lasso.items(), key = lambda x: x[1][1], reverse = True))
#                 print(sorted(self.dict_elasticnet.items(), key = lambda x: x[1][1], reverse = True))
#                 print(sorted(self.dict_huberregressor.items(), key = lambda x: x[1][1], reverse = True))
#                 print(sorted(self.dict_bayesianridge.items(), key = lambda x: x[1][1], reverse = True))
#                 print(sorted(self.dict_adaboost.items(), key = lambda x: x[1][1], reverse = True))
#                 print(sorted(self.dict_gradientboosting.items(), key = lambda x: x[1][1], reverse = True))
#                 print(sorted(self.dict_randomforest.items(), key = lambda x: x[1][1], reverse = True))

#                 [('GradientBoostingRegressor 23 0.5 -0.2', (0.9843, 0.9029)), ('GradientBoostingRegressor 65 0.15 -0.2', (0.992, 0.9006)), ('GradientBoostingRegressor 78 0.1 -0.2', (0.9926, 0.8972)), ('GradientBoostingRegressor 28 0.45 -0.2', (0.9858, 0.8956)), ('GradientBoostingRegressor 30 0.4 -0.2', (0.9867, 0.8954)), ('GradientBoostingRegressor 128 0.0 -0.2', (0.9925, 0.8928)), ('GradientBoostingRegressor 87 0.05 -0.2', (0.9926, 0.8927)), ('GradientBoostingRegressor 19 0.6 -0.2', (0.9808, 0.8924)), ('GradientBoostingRegressor 49 0.25 -0.2', (0.9914, 0.8916)), ('GradientBoostingRegressor 20 0.55 -0.2', (0.9809, 0.8916)), ('GradientBoostingRegressor 60 0.2 -0.2', (0.9923, 0.8911)), ('GradientBoostingRegressor 45 0.3 -0.2', (0.9909, 0.8861)), ('GradientBoostingRegressor 33 0.35 -0.2', (0.9886, 0.8817)), ('GradientBoostingRegressor 15 0.7 -0.2', (0.9574, 0.8696)), ('GradientBoostingRegressor 15 0.65 -0.2', (0.9574, 0.8691))]
        
#         self.results_dict2[list(filter(lambda x: x[0][0:5] == 'Ridge', self.results_dict1.items()))[0] = sorted(list(filter(lambda x: x[0][0:5] == 'Ridge', self.results_dict1.items())))
#         self.results_dict2[list(filter(lambda x: x[0][0:5] == 'Lasso', self.results_dict1.items()))[:1][0][0]] = list(filter(lambda x: x[0][0:5] == 'Lasso', self.results_dict1.items()))[:1][0][1]
#         self.results_dict2[list(filter(lambda x: x[0][0:10] == 'ElasticNet', self.results_dict1.items()))[:1][0][0]] = list(filter(lambda x: x[0][0:10] == 'ElasticNet', self.results_dict1.items()))[:1][0][1]
#         self.results_dict2[list(filter(lambda x: x[0][0:14] == 'HuberRegressor', self.results_dict1.items()))[:1][0][0]] = list(filter(lambda x: x[0][0:14] == 'HuberRegressor', self.results_dict1.items()))[:1][0][1]
#         self.results_dict2[list(filter(lambda x: x[0][0:13] == 'BayesianRidge', self.results_dict1.items()))[:1][0][0]] = list(filter(lambda x: x[0][0:13] == 'BayesianRidge', self.results_dict1.items()))[:1][0][1]
#         self.results_dict2[list(filter(lambda x: x[0][0:17] == 'AdaBoostRegressor', self.results_dict1.items()))[:1][0][0]] = list(filter(lambda x: x[0][0:17] == 'AdaBoostRegressor', self.results_dict1.items()))[:1][0][1]
#         self.results_dict2[list(filter(lambda x: x[0][0:25] == 'GradientBoostingRegressor', self.results_dict1.items()))[:1][0][0]] = list(filter(lambda x: x[0][0:25] == 'GradientBoostingRegressor', self.results_dict1.items()))[:1][0][1]
#         self.results_dict2[list(filter(lambda x: x[0][0:21] == 'RandomForestRegressor', self.results_dict1.items()))[:1][0][0]] = list(filter(lambda x: x[0][0:21] == 'RandomForestRegressor', self.results_dict1.items()))[:1][0][1]
#         print(self.results_dict2)
        
#         print(list(filter(lambda x: x[0][0:5] == 'Ridge', self.results_dict1.items())))
                
    
    def set_model(self, *args):
        if 'Ridge' in args:
            self.model = Ridge(max_iter = 500)
        elif 'Lasso' in args: 
            self.model = Lasso(max_iter = 500) 
        elif 'ElasticNet' in args:
            self.model = ElasticNet(max_iter = 500)
        elif 'HuberRegressor' in args:
            self.model = HuberRegressor(max_iter = 500)
        elif 'BayesianRidge' in args:
            self.model = BayesianRidge(n_iter = 500)
        elif 'AdaBoostRegressor' in args:
            self.model = AdaBoostRegressor()
        elif 'GradientBoostingRegressor' in args:
            self.model = GradientBoostingRegressor(n_estimators = 500)
        elif 'RandomForestRegressor' in args:
            self.model = RandomForestRegressor(n_estimators = 500)
        else:
            print("Please choose one of the supported sklearn Regressor objects")
       

f1 = Feature_Prep()
f1.dummify(combineddf, 'GradientBoostingRegressor', np.linspace(0,0.7,15), -0.2)
# f1.best_features(combineddf, 'GradientBoostingRegressor', np.linspace(0,0.7,15), -0.2)
        

/Users/philippeheitzmann/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 346117935866.3144, tolerance: 696659484.3571944
  positive)
/Users/philippeheitzmann/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 466476705122.1967, tolerance: 696659484.3571944
  positive)
/Users/philippeheitzmann/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 111484593371.0434, tolerance: 696659484.3571944
  positive)
/Users/philippeheitzmann/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWar

112


[('Ridge 128', (0.8928, 0.8742, 'Ridge', 128, 0.0)),
 ('Ridge 65', (0.8446, 0.8719, 'Ridge', 65, 0.15)),
 ('Ridge 87', (0.8652, 0.8704, 'Ridge', 87, 0.05)),
 ('Ridge 78', (0.8587, 0.8694, 'Ridge', 78, 0.1)),
 ('Ridge 60', (0.8432, 0.8621, 'Ridge', 60, 0.2)),
 ('Ridge 45', (0.8288, 0.8576, 'Ridge', 45, 0.3)),
 ('Ridge 49', (0.8324, 0.8573, 'Ridge', 49, 0.25)),
 ('Ridge 33', (0.8128, 0.8403, 'Ridge', 33, 0.35)),
 ('Ridge 30', (0.8076, 0.8359, 'Ridge', 30, 0.4)),
 ('Ridge 28', (0.7993, 0.8298, 'Ridge', 28, 0.45)),
 ('Ridge 23', (0.7857, 0.8062, 'Ridge', 23, 0.5)),
 ('Ridge 20', (0.7832, 0.8056, 'Ridge', 20, 0.55)),
 ('Ridge 19', (0.7828, 0.8052, 'Ridge', 19, 0.6)),
 ('Ridge 15', (0.7576, 0.7843, 'Ridge', 15, 0.7)),
 ('RandomForestRegressor 60',
  (0.9789, 0.8949, 'RandomForestRegressor', 60, 0.2)),
 ('RandomForestRegressor 49',
  (0.9785, 0.8944, 'RandomForestRegressor', 49, 0.25)),
 ('RandomForestRegressor 20',
  (0.9752, 0.8928, 'RandomForestRegressor', 20, 0.55)),
 ('RandomForestRegres

In [6]:
str1 = 'RandomForestRegressor 128 0.0 -0.2'
str1[0:21]

'RandomForestRegressor'

In [ ]:
x1.set_model('Lasso')

In [40]:
#child class of Feature_Prep
#fitcheck method allows to select feature subsets and fits these with different models provided in self.models 
class FeatureSelection(Feature_Prep): 

    def __init__(self):
        super().__init__()
        self.poscutoff = 0.5
        self.negcutoff = -0.5
        self.featurenames = None
        self.model = Ridge()
        self.models = ['Ridge', 'Lasso', 'ElasticNet', 'BayesianRidge', 'DecisionTreeRegressor', 'AdaBoostRegressor', 'GradientBoostingRegressor', 'RandomForestRegressor']
        self.trainsubset = None      #trainhsubset                     #is there a better way to initialize these self attributes?
        self.testsubset = None       #testhsubset
        self.testpredictions = None
        self.x_train = None          #x_train of trainhsubset
        self.x_test = None           #x_test of trainhsubset
        self.y_train = None          #y_train of target
        self.y_test = None           #y_test of target
        self.localtime = time.asctime( time.localtime(time.time()) )
        self.resultsdict = {}
        self.resultsdict1 = {}
        self.poscorrinterval = list(np.linspace(0,.75,10))
    
  
    def find_features(self, X, poscutoff, negcutoff):    #X is your combineddf
        self.dummify(X)
        self.poscutoff = poscutoff
        self.negcutoff = negcutoff
        X0 = self.trainingdf.corr().iloc[:,-1]
        dict1 = {}
        for idx, i in enumerate(X0): 
            if i > poscutoff:
                dict1[X0.index[idx]] = i
            elif i < negcutoff:
                dict1[X0.index[idx]] = i
            else: 
                continue
        self.featurenames = [x[0] for x in list(filter(lambda x: x[0] != 'SalePrice', dict1.items()))]
        self.trainsubset, self.testsubet = self.trainingdf[self.featurenames], self.testingdf[self.featurenames]        
        
    def train_test_split(self, Y, size):   #Y is your target y variable pd Series 
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.trainsubset, Y, test_size = size)
#         print('x_train NAs are',self.x_train.isna().sum()[200:], 'y_train NAs are', self.y_train.isna().sum())
    
    def fitcheck(self, X, Y, size):       #X is your combineddf, Y is your target y variable pd Series
    
        for model in self.models:
            
            self.set_model(model)
            
            for i in self.poscorrinterval:
                self.find_features(X, poscutoff = i, negcutoff = -0.5)
                self.train_test_split(Y, size) 
                self.model.fit(self.x_train, self.y_train)
                key1 = model + ' ' + str(len(self.featurenames)) + ' ' + str(i)
                self.resultsdict1[key1] = [np.round(self.model.score(self.x_train, self.y_train),3), np.round(self.model.score(self.x_test, self.y_test),3)]
                
        return sorted(self.resultsdict1.items(), key = lambda x: x[1][1], reverse = True)

    
    def set_model(self, *args):
        if 'Ridge' in args:
            self.model = Ridge()
            return self.model
        elif 'Lasso' in args: 
            self.model = Lasso() 
            return self.model
        elif 'ElasticNet' in args:
            self.model = ElasticNet()
            return self.model
        elif 'HuberRegressor' in args:
            self.model = HuberRegressor()
            return self.model
        elif 'BayesianRidge' in args:
            self.model = BayesianRidge()
            return self.model
        elif 'DecisionTreeRegressor' in args:   #may make sense to not include 
            self.model = DecisionTreeRegressor()
            return self.model
        elif 'AdaBoostRegressor' in args:
            self.model = AdaBoostRegressor()
            return self.model
        elif 'GradientBoostingRegressor' in args:
            self.model = GradientBoostingRegressor()
            return self.model
        elif 'RandomForestRegressor' in args:
            self.model = RandomForestRegressor()
            return self.model
        else:
            return self.model
    
    def fit(self, X, Y):     #Y is your target y variable pd Series 
        self.model.fit(X, Y)
    
    
    def predict(self):    
        self.testpredictions = self.model.predict(self.testsubset)
    
    #static method declarators - don't need access to methods Look into function scope 
#     @staticmethod

    def preprocess(self, X, Y):   #Y is your testh dataset, X is your trainh dataset
        if isinstance(X, list): raise TypeError("X needs to be a DataFrame object")
        elif isinstance(X, np.ndarray): raise TypeError()
#         X = X.fillna(X.mean())    #Q: would this work? Why not?
#         print(X.isna().sum(),'\n')
#         Y = Y.fillna(Y.mean())
#         print(Y.isna().sum())
        self.trainsubset = X.fillna(X.mean())
        self.testsubset = Y.fillna(Y.mean()) 
        print(self.testsubset.isna().sum())
#         print(self.testsubset.isna().sum())
        
        
    def to_csv1(self):
        testpredictionsdf = pd.DataFrame(self.testpredictions, index = range(1461, 2920), columns = ['SalePrice'])
        testpredictionsdf.index.name = 'Id'
        testpredictionsdf.to_csv('Kaggle Submission %s .csv'%(self.localtime))


x1 = FeatureSelection() 
x1.fitcheck(combineddf, target, 0.2)
# x1.find_features(combineddf, 0.5, -0.5)


TypeError: dummify() missing 3 required positional arguments: 'model', 'posinterval', and 'negcutoff'

In [ ]:
x1 = FeatureSelection()
x1.df1

In [100]:
#child class of FeatureSelection
#sets up hyperparameter tuning for the best models 

class KaggleOutput(FeatureSelection):
    
    def __init__(self):
        super().__init__()
        self.paramgrid = {}
        self.df3 = None  
        self.dict3 = {}
        
    def prep(self, X, model, cutoff, negcutoff):
        self.target = X.iloc[0:1460, -1]
        self.type_filter(X)
        for feature in self.objectlist:
            self.df3 = pd.concat([self.df3, pd.get_dummies(X[feature], prefix = feature, drop_first=True)], axis = 1)
        for i in list(filter(lambda x: x[1] == 'int64' or x[1] == 'float64', self.featuredict.items())):
            try:
                self.df3 = pd.concat([self.df3, X[i[0]]], axis = 1)
            except:
                continue
        self.trainingdf = self.df3.iloc[0:1460,1:]
        self.testingdf = self.df3.iloc[1460:,1:-1]
        self.trainingdf = self.trainingdf.fillna(self.trainingdf.mean())
        self.testingdf = self.testingdf.fillna(self.testingdf.mean())
        
        self.set_model(model)
        
        for idx, i in enumerate(self.trainingdf.corr().iloc[:,-1]):   #could just make dict3 into a list 
            if i > cutoff or i < negcutoff:
                self.dict3[self.trainingdf.columns[idx]] = i
            else:
                continue
        self.feature_name_list = [x[0] for x in self.dict3.items() if x[0] != 'SalePrice']
        self.trainingsubset = self.trainingdf[self.feature_name_list]
        self.testingsubset = self.testingdf[self.feature_name_list]

        self.model.fit(self.trainingsubset, self.target)
        
        predictiondf = pd.DataFrame(self.model.predict(self.trainingsubset), index = range(0, 1460), columns = [str(self.model)])
#         predictiondf.index.name = 'Id'
        predictiondf.to_csv('Kaggle Submission TrueTrain %s .csv'%(str(self.model) + ' ' + str(len(self.feature_name_list)) + ' ' + self.localtime))
        
    def kaggle_output(self, X, Y, Y2, model, paramgrid, poscutoff, negcutoff):    #Y is target, Y2 is testh
        
        self.prep(combineddf)
        
        self.set_model(model)
        try: 
            self.model.set_params(paramgrid)
        except:
            pass
        self.find_features(X, Y2, poscutoff, negcutoff)
        self.fit(self.trainsubset, Y)
        self.predict()
        self.to_csv1()
        
    def tuning(self, model, X, Y, Y2, paramgrid, poscutoff, negcutoff, size):    #Y is target, Y2 is testh
        self.set_model(model)
        self.find_features(X, Y2, poscutoff, negcutoff)
        self.train_test_split(Y, size)
        grid_search1 = GridSearchCV(self.model, paramgrid, scoring = 'neg_root_mean_squared_error', n_jobs = -1, cv = 5)
        grid_search1.fit(self.x_train, self.y_train)   #cross-fold validation - check for overfitting post tuning 
        self.paramgrid = grid_search1.best_params_
        print(self.paramgrid)
        print('The training error is %.4f' %(1 - grid_search1.best_estimator_.score(self.x_train, self.y_train)))
        print('The testing error is %.4f' %(1 - grid_search1.best_estimator_.score(self.x_test, self.y_test)))
        self.kaggle_output(X, Y, Y2, model, paramgrid, poscutoff, negcutoff)
        
        
x2 = KaggleOutput()
# x2.kaggle_output(trainh, target, testh, 'GradientBoostingRegressor', 0.23275862068965517, -0.5)
# x2.kaggle_output(trainh, target, testh, 'RandomForestRegressor', 0.33620689655172414, -0.5)
# x2.kaggle_output(trainh, target, testh, 'RandomForestRegressor', 0.1810344827586207, -0.5)
# x2.tuning('GradientBoostingRegressor', trainh, target, testh, {'learning_rate':np.linspace(0.01,0.1,10), 'subsample':np.linspace(0.3,1,5)}, 0.23275862068965517, -0.5, 0.2)
x2.prep(combineddf, 'ElasticNet', 0.130000, -0.2)



/Users/philippeheitzmann/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 925125760996.3031, tolerance: 920791133.4609975
  positive)


In [4]:
#Training data predictions 
#loading the predictions of the eight different regressors for the training data

ada_boostdf2 = pd.read_csv('Kaggle Submission TrueTrain AdaBoostRegressor() 41 Sat Sep 19 19:49:02 2020 .csv', sep = ',')
ada_boostdf2 = ada_boostdf2.drop(['Unnamed: 0'], axis = 1)

bayesian_ridgedf2 = pd.read_csv('Kaggle Submission TrueTrain BayesianRidge() 84 Sat Sep 19 19:48:45 2020 .csv', sep = ',')
bayesian_ridgedf2 = bayesian_ridgedf2.drop(['Unnamed: 0'], axis = 1)

elastic_netdf2 = pd.read_csv('Kaggle Submission TrueTrain ElasticNet() 68 Sat Sep 19 19:50:37 2020 .csv', sep = ',')
elastic_netdf2 = elastic_netdf2.drop(['Unnamed: 0'], axis = 1)

gradient_boosting_regressordf2 = pd.read_csv('Kaggle Submission TrueTrain GradientBoostingRegressor() 30 Sat Sep 19 19:48:23 2020 .csv', sep = ',')
gradient_boosting_regressordf2 = gradient_boosting_regressordf2.drop(['Unnamed: 0'], axis = 1)

huber_regressordf2 = pd.read_csv('Kaggle Submission TrueTrain HuberRegressor() 33 Sat Sep 19 19:48:08 2020 .csv', sep = ',')
huber_regressordf2 = huber_regressordf2.drop(['Unnamed: 0'], axis = 1)

lassodf2 = pd.read_csv('Kaggle Submission TrueTrain Lasso() 78 Sat Sep 19 19:47:58 2020 .csv', sep = ',')
lassodf2 = lassodf2.drop(['Unnamed: 0'], axis = 1)

random_forestdf2 = pd.read_csv('Kaggle Submission TrueTrain RandomForestRegressor() 46 Sat Sep 19 19:47:49 2020 .csv', sep = ',')
random_forestdf2 = random_forestdf2.drop(['Unnamed: 0'], axis = 1)

ridgedf2 = pd.read_csv('Kaggle Submission TrueTrain Ridge() 84 Sat Sep 19 19:47:37 2020 .csv', sep = ',')
ridgedf2 = ridgedf2.drop(['Unnamed: 0'], axis = 1)



In [ ]:
#Testing data
#loading the predictions of the eight different regressors for the testing data

ada_boostdf = pd.read_csv('Kaggle Submission AdaBoostRegressor() 41 Sat Sep 19 19:08:46 2020 .csv', sep = ',')
ada_boostdf = ada_boostdf.drop(['Id'], axis = 1)
ada_boostdf.columns = ['AdaBoost']

bayesian_ridgedf = pd.read_csv('Kaggle Submission BayesianRidge() 84 Sat Sep 19 19:08:34 2020 .csv', sep = ',')
bayesian_ridgedf = bayesian_ridgedf.drop(['Id'], axis = 1)
bayesian_ridgedf.columns = ['BayesianRidge']

elastic_netdf = pd.read_csv('Kaggle Submission ElasticNet() 68 Sat Sep 19 19:08:18 2020 .csv', sep = ',')
elastic_netdf = elastic_netdf.drop(['Id'], axis = 1)
elastic_netdf.columns = ['ElasticNet']

gradient_boosting_regressordf = pd.read_csv('Kaggle Submission GradientBoostingRegressor() 30 Sat Sep 19 19:08:05 2020 .csv', sep = ',')
gradient_boosting_regressordf = gradient_boosting_regressordf.drop(['Id'], axis = 1)
gradient_boosting_regressordf.columns = ['GradientBoostingR']

huber_regressordf = pd.read_csv('Kaggle Submission HuberRegressor() 33 Sat Sep 19 19:07:49 2020 .csv', sep = ',')
huber_regressordf = huber_regressordf.drop(['Id'], axis = 1)
huber_regressordf.columns = ['HuberRegressor']

lassodf = pd.read_csv('Kaggle Submission Lasso() 78 Sat Sep 19 19:06:58 2020 .csv', sep = ',')
lassodf = lassodf.drop(['Id'], axis = 1)
lassodf.columns = ['Lasso']

random_forestdf = pd.read_csv('Kaggle Submission RandomForestRegressor() 46 Sat Sep 19 19:06:48 2020 .csv', sep = ',')
random_forestdf = random_forestdf.drop(['Id'], axis = 1)
random_forestdf.columns = ['RandomForest']

ridgedf = pd.read_csv('Kaggle Submission Ridge() 84 Sat Sep 19 19:06:14 2020 .csv', sep = ',')
ridgedf = ridgedf.drop(['Id'], axis = 1)
ridgedf.columns = ['Ridge']

In [16]:
lm2.fit(x_train, y_train)

NameError: name 'lm2' is not defined

In [5]:
#concatenating the training data predictions 
merged_train_df = pd.concat([ada_boostdf2, bayesian_ridgedf2, elastic_netdf2, gradient_boosting_regressordf2, huber_regressordf2, lassodf2, random_forestdf2, ridgedf2], axis = 1)
merged_train_df.columns=['AdaBoost','BayesianRidge','ElasticNet','GradientBoostingR','HuberRegressor','Lasso','RandomForest','Ridge']
merged_train_df

#splitting our data 

x_train, x_test, y_train, y_test = train_test_split(merged_train_df, target, test_size = 0.33)


,AdaBoost,BayesianRidge,ElasticNet,GradientBoostingR,HuberRegressor,Lasso,RandomForest,Ridge
0,209918.558219,204643.338933,216274.894771,200929.126639,205533.606866,200724.033256,205928.25,204452.286881
1,157568.463035,170243.370892,172925.990277,160553.412684,177892.851168,168976.299913,176221.50,170509.050471
2,215471.223097,210530.840398,223279.154200,215751.855269,214515.376684,206468.171216,219537.00,209515.089784
3,196621.649606,198401.512814,175824.875640,181124.727045,156037.880122,175662.106874,162009.99,199414.177471
4,286448.713580,305825.972255,286534.155457,290443.695034,277767.912231,313897.232469,263430.93,309958.711713
...,...,...,...,...,...,...,...,...
1455,180347.611111,168418.138700,179646.372317,174747.795139,181618.083769,170597.607264,175209.00,167732.479074
1456,213311.468619,189975.492778,228493.732566,205351.458569,240614.929895,186991.246856,207386.98,187724.860539
1457,248416.982301,254208.251693,225727.252185,263583.257529,222303.285810,238559.054166,260710.79,258415.510399
1458,132523.478261,157429.068970,143717.549075,137584.995036,124541.886324,161246.286058,135171.90,157394.194728


In [17]:
#using simple Linear REgression to fit the train subset of the merged training predictions and seeing how these fare vs testing subset of training dataste  
lm2 = LinearRegression()
lm2.fit(x_train, y_train)
print(lm2.score(x_train, y_train), lm2.score(x_test, y_test))
merged_test_predictionsdf2 = pd.DataFrame(lm2.predict(merged_test_df), index = range(1461, 2920), columns = ['SalePrice'])
merged_test_predictionsdf2.index.name = 'Id'
merged_test_predictionsdf2.to_csv('Merged_test_predictions3.csv', sep=',')


0.9904595708265973 0.9827508181125693


In [8]:
#tuning our GradientBoostingRegressor with GridSearchCV to find the best parameters on our training subset 
gbr2 = GradientBoostingRegressor()
#'max_depth':range(2,31),
paramgrid = {'learning_rate':np.linspace(0.5,1,5), 'n_estimators':range(100,500,100)}
gridsearch_gbr2 = GridSearchCV(gbr2, paramgrid, cv = 5, n_jobs = -1, scoring = 'neg_root_mean_squared_error')
gridsearch_gbr2.fit(x_train, y_train)
print(gridsearch_gbr2.best_params_)


# gbr2.fit(x_train, y_train)
# print(gbr2.score(x_train, y_train), gbr2.score(x_test, y_test))



{'learning_rate': 0.5, 'n_estimators': 100}


In [10]:
print(gridsearch_gbr2.score(x_train, y_train),gridsearch_gbr2.score(x_test, y_test))   #is 5x root MSE cause for concern?

-2106.1344174013225 -10411.41642229556


In [9]:
gridsearch_gbr2.best_estimator_

GradientBoostingRegressor(learning_rate=0.5)

In [13]:
#trying the best parameters to fit a new Gradient Boosting Regressor
gbr3 = GradientBoostingRegressor(learning_rate = 0.5, n_estimators = 100)
gbr3.fit(x_train, y_train)
print(gbr3.score(x_train, y_train),gbr3.score(x_test,y_test))

0.9993187332018426 0.9753398303784183


In [18]:
#using the model we training on the training subsets of the stacked training predictions
merged_test_df = pd.concat([ada_boostdf, bayesian_ridgedf, elastic_netdf, gradient_boosting_regressordf, huber_regressordf, lassodf, random_forestdf, ridgedf], axis = 1)
merged_test_predictionsdf = pd.DataFrame(gbr3.predict(merged_test_df), index = range(1461, 2920), columns = ['SalePrice'])
merged_test_predictionsdf.index.name = 'Id'
merged_test_predictionsdf.to_csv('Merged_test_predictions4.csv', sep=',')

,AdaBoost,BayesianRidge,ElasticNet,GradientBoostingR,HuberRegressor,Lasso,RandomForest,Ridge
0,126932.846608,96827.588553,120181.717252,120231.346171,155100.668744,89258.417607,124882.16,93178.034492
1,146598.964851,150417.215895,162485.050339,148107.921691,173032.379790,150328.550023,152736.50,147853.325946
2,176805.524664,175330.108981,189139.983827,169918.584003,193099.998103,176456.835426,179665.00,174683.594110
3,183788.884058,189143.331491,202938.982581,185881.896019,187942.234484,189812.127414,187808.05,187157.692481
4,228829.474954,223954.918480,192174.540561,201404.985474,173803.388146,236451.029484,205201.56,231186.243125
...,...,...,...,...,...,...,...,...
1454,118394.975078,74444.864119,76016.830582,74796.164090,84081.864902,73949.005111,82048.61,74527.736377
1455,118394.975078,80671.553692,89552.805055,86043.686889,116544.448907,80574.049128,84768.50,79957.929065
1456,155459.591304,170812.633689,176303.053845,153504.605641,186586.926183,171902.735894,153055.50,171132.100356
1457,136101.666667,107808.812720,118634.877237,121668.893824,101713.785216,107520.684867,109072.00,105205.493180
